In [1]:
import urllib.request
import os
import zipfile
import pandas as pd
from pandas import DataFrame
import geopandas as gpd

In [2]:
mesh1_path = 'I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/mesh_for_work/mesh1_code/mesh1.csv'
mesh1_code = pd.read_csv(mesh1_path)

mesh1_code = pd.DataFrame(mesh1_code, columns=['Name'])

In [70]:
# 平成28年のデータの読み込み
base_url = "https://nlftp.mlit.go.jp/ksj/gml/data/L03-b/L03-b-16/L03-b-16_"

# mesh1_codeが前のRのコードの結果として得られるリストまたは配列と仮定します
# mesh1_codeの値を使ってURLを組み立て
url_list = [base_url + str(code) + "-jgd_GML.zip" for code in mesh1_code['Name']]

In [72]:
disable_mesh1_list = []

for i, url in enumerate(url_list):
    # 保存ファイル名の指定
    destfile = f"I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/mesh_for_work/100m_mesh_zip/landuse_mesh_{mesh1_code['Name'][i]}.zip"
    
    # ダウンロード（データがない場合はスキップ）
    try:
        urllib.request.urlretrieve(url, destfile)
    except Exception as e:
        print(f"{url} did not work out due to {e}")
        disable_mesh1_list.append(url)
        
print(disable_mesh1_list)

https://nlftp.mlit.go.jp/ksj/gml/data/L03-b/L03-b-16/L03-b-16_6740-jgd_GML.zip did not work out due to HTTP Error 404: Not Found
['https://nlftp.mlit.go.jp/ksj/gml/data/L03-b/L03-b-16/L03-b-16_6740-jgd_GML.zip']


In [73]:
# zipファイルパスの取得
zip_path = 'I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/mesh_for_work/100m_mesh_zip'
ziplist = [os.path.join(zip_path, f) for f in os.listdir(zip_path) if f.endswith('.zip')]

# 保存ファイル名用
mesh_file = 'I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/mesh_for_work/100m_mesh_shpall'
filename = [f for f in os.listdir(mesh_file) if f.endswith('.zip')]

# zipファイルの解凍
for i, zip_path in enumerate(ziplist):
    # zipファイルの解凍
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(mesh_file)

OSError: [Errno 22] Invalid argument

In [75]:
# shpファイルのフルパスを作成
root_path = "I:\マイドライブ\水道管劣化予測\水道管劣化予測\src\国土数値情報\土地利用メッシュ\mesh_for_work"
shp_list = []
for dirpath, dirnames, filenames in os.walk(root_path):
    for filename in filenames:
        if filename.endswith('.shp'):
            shp_list.append(os.path.join(dirpath, filename))

# shpファイルの読み込み
dataframes = []
for shp_path in shp_list:
    dataframes.append(gpd.read_file(shp_path, encoding='shift-jis'))

In [78]:
# マージ
national_100m_mesh = gpd.GeoDataFrame(columns=['メッシュ', '土地利用種', '撮影年月日', 'geometry'])

disable_mesh_gdf = []
for gdf in dataframes:
    #座標系の変換
    dst_proj = 4326  # 変換後の座標系を指定
    gdf = gdf.to_crs(epsg=dst_proj)  # 変換後座標に変換
    if set(['メッシュ', '土地利用種', '撮影年月日', 'geometry']).issubset(gdf.columns):
        national_100m_mesh = pd.concat([national_100m_mesh, gdf], ignore_index=True, verify_integrity=True)
        
    else:
        print(f"{gdf} does not have the required columns and will not be merged.")
        disable_mesh_gdf.append(gdf)
        
# # データの保存
# national_100m_mesh.to_file("path/merge.shp")

        Name Descriptio                                           geometry
0     303650     303650  POLYGON ((136.00000 20.41667, 136.00000 20.500...
1     362257     362257  POLYGON ((122.87500 24.41667, 122.87500 24.500...
2     362350     362350  POLYGON ((123.00000 24.41667, 123.00000 24.500...
3     362324     362324  POLYGON ((123.50000 24.16667, 123.50000 24.250...
4     362335     362335  POLYGON ((123.62500 24.25000, 123.62500 24.333...
...      ...        ...                                                ...
4884  684816     684816  POLYGON ((148.75000 45.41667, 148.75000 45.500...
4885  684826     684826  POLYGON ((148.75000 45.50000, 148.75000 45.583...
4886  684807     684807  POLYGON ((148.87500 45.33333, 148.87500 45.416...
4887  684817     684817  POLYGON ((148.87500 45.41667, 148.87500 45.500...
4888  684827     684827  POLYGON ((148.87500 45.50000, 148.87500 45.583...

[4889 rows x 3 columns] does not have the required columns and will not be merged.
        OBJECTID

In [ ]:
pref_polygon_path = 'I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/mesh_for_work/prefecture_polygon/prefecture.geojson'
pref_polygon = gpd.read_file(pref_polygon_path)

i = 1

for index, pref in pref_polygon.iterrows():
    
    # GeoDataFrameを作成し、'geometry'カラムを設定します
    gdf_pref = gpd.GeoDataFrame(geometry=[pref['geometry']])
    
    dst_proj = 4326  # 変換後の座標系を指定
    gdf_pref = gdf_pref.to_crs(epsg=dst_proj)  # 変換後座標に変換
    
    pref_100m_mesh = gpd.sjoin(gdf_pref, national_100m_mesh, op='intersects')
    #都道府県のgeometryは必要ないので削除
    pref_100m_mesh = pref_100m_mesh[['メッシュ', '土地利用種', '撮影年月日']]


    pref_100m_mesh = pd.merge(pref_100m_mesh, national_100m_mesh[['メッシュ', 'geometry']], on='メッシュ')
    
    print(f'--都道府県コード：{i}の出力中--')
    weather_1km_path = f'I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/100m_mesh_全国分/{i}_100m_mesh.geojson'
    gpd.GeoDataFrame(pref_100m_mesh, geometry='geometry').to_file(weather_1km_path)
    print(f'--都道府県コード：{i}の出力完了--')
    
    i += 1

In [83]:
kagawa_100m_mesh_path = 'I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/100m_mesh_全国分/37_100M_mesh.geojson'
kagawa_100m_mesh = gpd.read_file(kagawa_100m_mesh_path)

,メッシュ,土地利用種,撮影年月日,geometry
0,3036400000,1500,20171004,"POLYGON ((136.00000 20.33333, 136.00000 20.334..."
1,3036400001,1500,20171004,"POLYGON ((136.00125 20.33333, 136.00125 20.334..."
2,3036400002,1500,20171004,"POLYGON ((136.00250 20.33333, 136.00250 20.334..."
3,3036400003,1500,20171004,"POLYGON ((136.00375 20.33333, 136.00375 20.334..."
4,3036400004,1500,20171004,"POLYGON ((136.00500 20.33333, 136.00500 20.334..."
...,...,...,...,...
37819995,6240779995,1500,20160529,"POLYGON ((140.99375 41.99917, 140.99375 42.000..."
37819996,6240779996,1500,20160529,"POLYGON ((140.99500 41.99917, 140.99500 42.000..."
37819997,6240779997,1500,20160529,"POLYGON ((140.99625 41.99917, 140.99625 42.000..."
37819998,6240779998,1500,20160529,"POLYGON ((140.99750 41.99917, 140.99750 42.000..."


In [85]:
# データの保存
national_100m_mesh_path = 'I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/mesh_for_work/national_100m_mesh.gpkg'
national_100m_mesh.to_file(national_100m_mesh_path)

CPLE_AppDefinedError: b'sqlite3_exec(ROLLBACK) failed: cannot rollback - no transaction is active'

Exception ignored in: 'fiona.ogrext.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona\_err.pyx", line 198, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(ROLLBACK) failed: cannot rollback - no transaction is active'
